In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/ml_matrix'

/content/drive/My Drive/Colab Notebooks/ml_matrix


In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
df['manufacturer_cat'] = df['manufacturer'].map(lambda x: str(x).lower()).factorize()[0]
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [21]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [22]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [33]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [34]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [35]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_part number', 'feat_fabric content', 'feat_size',
       'feat_wheel type', 'feat_product #', 'feat_item width',
       'feat_metal type', 'feat_case tone', 'feat_number of pieces',
       'feat_alarm'],
      dtype='object', length=527)

In [45]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

{k: v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer_part_number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
# feats_cat = [x for x in df.columns if 'cat' in x]
# feats += feats_cat

# feats = list(set(feats))

In [54]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].shape

(8846, 2)

In [0]:
result = run_model(feats, model)

In [85]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
model.fit(X, y)

perm = PermutationImportance(model, random_state=1).fit(X, y)
print(result)
eli5.show_weights(perm, feature_names=feats)


(-57.17319916647938, 4.240863641857993)


Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_gender_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [64]:
df ['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
hickey freeman    0.000055
dkny              0.000055
swiss cross       0.000055
extreme storm     0.000055
ouray             0.000055
Name: brand, Length: 1732, dtype: float64

In [70]:
df [ df[ 'brand' ] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'pattern': 'geometric', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'gender': 'men', 'brand': 'nike', 'color': 'court purple/purple venom/white/metallic silver'},
       {'style': 'cushion no-show socks', 'material': 'nylon', 'country/region of manufacture': 'thailand', 'condition': 'new with tags'},
       {'style': 'basketball shoes', 'pattern': 'zoom terminator cuse', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/action red/team red', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '707552060', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [87]:
ls

data/  day3.ipynb  LICENSE  README.md  ShoesPricesML.ipynb
